In [1]:
import warnings
warnings.filterwarnings("ignore")
import csv
import numpy as np
import pandas as pd
from importlib import reload
from os import listdir
from os.path import isfile, join
from model.tokenizer import DostoevskyTokenizer

unable to import 'smart_open.gcs', disabling that module
Using TensorFlow backend.


In [7]:
PATH_TO_DATASET = "..\..\..\public_youtube1120_mp3\public_youtube1120\\"
PATH_TO_CSV = "..\..\..\\public_youtube1120_mp3.csv"

In [ ]:
from utils.walker import get_csv_for_dataset
get_csv_for_dataset(PATH_TO_DATASET, PATH_TO_CSV, verbose=False)

In [ ]:
# initial dataset
init_ds = pd.read_csv("..\..\\public_youtube1120_mp3.csv", encoding='Windows-1251')

In [8]:
init_ds.head()

,path,text
0,.\public_youtube1120\0\00\000f45f0543c.txt,вань давай больше никогда не расставаться
1,.\public_youtube1120\0\00\016f321759c5.txt,они под приборную панель устанавливаются
2,.\public_youtube1120\0\00\03010c1ba92d.txt,что там с пассажирами яхты все девочки из эско...
3,.\public_youtube1120\0\00\0373ac3dedfd.txt,я не сплю что случилось
4,.\public_youtube1120\0\00\03da8e2e247a.txt,возвращайтесь в город и обязательно сходите к ...


In [9]:
init_ds.describe()

,path,text
count,1410911,1410911
unique,1410911,1227753
top,.\public_youtube1120\a\c6\6e7b347d8617.txt,да
freq,1,4421


In [19]:
PATH_TO_RESULT = '..\\results\\'
FOLDER_NAME_IN = 'results_by100kstep\\'
FOLDER_NAME_OUT = 'results_after_scalling\\'

In [ ]:
model = DostoevskyTokenizer()
model.load_main_file(PATH_TO_CSV)
model.init_tokenizer()

In [ ]:
model.get_files_with_predict(path_to_save=FOLDER_NAME_IN)

In [20]:
result_files = [f for f in listdir(PATH_TO_RESULT + FOLDER_NAME_IN) if isfile(join(PATH_TO_RESULT + FOLDER_NAME_IN, f))]

In [21]:
result_files

['result0-100000.csv',
 'result100000-200000.csv',
 'result200000-300000.csv',
 'result300000-400000.csv',
 'result400000-500000.csv',
 'result500000-600000.csv',
 'result600000-700000.csv',
 'result700000-800000.csv',
 'result800000-900000.csv',
 'result900000-1000000.csv']

Number of samples with neutral emotions

In [28]:
res_ds = pd.read_csv(PATH_TO_RESULT + FOLDER_NAME_IN + 'result100000-200000.csv', encoding='Windows-1251', index_col=0)

In [ ]:
res_ds.head()

In [158]:
labels = [
        'positive',
        'neutral',
        'negative',
        'skip',
        'speech',
        'unknown',
        ]

def f(x):
    global count_neut_before
    index_first_max = int(x.nlargest(1).idxmin())
    index_second_max = int(x.nlargest(2).idxmin())
    if labels[index_first_max] == 'neutral':
        count_neut_before = count_neut_before + 1
        if x[index_second_max]/x[index_first_max] > x[index_first_max]:
            t = x[index_second_max]
            x[index_second_max] = x[index_first_max]
            x[index_first_max] = t
            return index_second_max
        else:
            return index_first_max
    return index_first_max
        #print('x: ', x, 'imax:', typeindex_first_max))

for csv_name in result_files:
    count_neut_before = 0
    df_res = pd.read_csv(PATH_TO_RESULT + FOLDER_NAME_IN + csv_name)
    df_res['emotions'] = df_res[['0', '1', '2', '3', '4', '5']].T.apply(lambda x: f(x))
    df_res['emotions'] = df_res['emotions'].T.apply(lambda x: labels[int(x)])
    count_neut_after = df_res[df_res.emotions == 'neutral'].emotions.count()
    print("neutral emot. in {} before scalling: {}, after: {}, delta: {}".format(csv_name, count_neut_before, count_neut_after, count_neut_before - count_neut_after))
    df_res.to_csv(PATH_TO_RESULT + FOLDER_NAME_OUT + 'scalling_' + csv_name, encoding='Windows-1251')

neutral emot. in result0-100000.csv before scalling: 90440, after: 75898, delta: 14542
neutral emot. in result100000-200000.csv before scalling: 90331, after: 75790, delta: 14541
neutral emot. in result200000-300000.csv before scalling: 90382, after: 75953, delta: 14429
neutral emot. in result300000-400000.csv before scalling: 90331, after: 75661, delta: 14670
neutral emot. in result400000-500000.csv before scalling: 90392, after: 75801, delta: 14591
neutral emot. in result500000-600000.csv before scalling: 90471, after: 75975, delta: 14496
neutral emot. in result600000-700000.csv before scalling: 90325, after: 75687, delta: 14638
neutral emot. in result700000-800000.csv before scalling: 90501, after: 75948, delta: 14553
neutral emot. in result800000-900000.csv before scalling: 90576, after: 75935, delta: 14641
neutral emot. in result900000-1000000.csv before scalling: 90301, after: 75633, delta: 14668


In [8]:
from utils import tools
import importlib
tools = importlib.reload(tools)

PATH_TO_CSV_INITIAL = '..\\results\\results_by100kstep\\'
PATH_TO_CSV_SCALLED = '..\\results_after_scalling\\'

tools.get_stat_from_initial_results(PATH_TO_CSV_INITIAL)

,neutral,speech,skip,negative,positive,unknown
count,904050,11606,10160,40317,29740,4127
std,0.144263,0.206007,0.156701,0.158562,0.112835,0.0527393
25%,0.531548,0.509873,0.361025,0.404409,0.387019,0.51033
50%,0.642497,0.736593,0.422581,0.492807,0.445975,0.51033
75%,0.74641,0.883418,0.524953,0.624461,0.531599,0.51033


In [200]:
tools.get_stat_from_initial_results(PATH_TO_CSV_SCALLED)

,neutral,speech,skip,negative,positive,unknown
count,758281,14156,45271,99334,71750,11208
std,0.116796,0.247594,0.0956843,0.166778,0.118162,0.142757
25%,0.589792,0.382231,0.341921,0.2634,0.27846,0.201188
50%,0.674712,0.617436,0.4015,0.338735,0.34075,0.232169
75%,0.765649,0.857464,0.44352,0.45745,0.424372,0.51033


In [65]:
result_files_after_scalling = [f for f in listdir(PATH_TO_CSV_SCALLED) if isfile(join(PATH_TO_CSV_SCALLED, f))]
result_files_after_scalling

['results_after_scallingscalling_result0-100000.csv',
 'results_after_scallingscalling_result100000-200000.csv',
 'results_after_scallingscalling_result200000-300000.csv',
 'results_after_scallingscalling_result300000-400000.csv',
 'results_after_scallingscalling_result400000-500000.csv',
 'results_after_scallingscalling_result500000-600000.csv',
 'results_after_scallingscalling_result600000-700000.csv',
 'results_after_scallingscalling_result700000-800000.csv',
 'results_after_scallingscalling_result800000-900000.csv',
 'results_after_scallingscalling_result900000-1000000.csv']

In [68]:
import os
from shutil import copy
from utils import tools
import importlib
tools = importlib.reload(tools)

PATH_TO_DATASET_SOURCE = "..\..\public_youtube1120_mp3\\"
PATH_TO_DATASET_DESTINATION = "..\..\dataset_based_p_y1120_mp3"
    
tools.get_train_val(PATH_TO_CSV_SCALLED + result_files_after_scalling[5], PATH_TO_DATASET_SOURCE, PATH_TO_DATASET_DESTINATION)

Directory ..\..\dataset_based_p_y1120_mp3\train already exist
Directory ..\..\dataset_based_p_y1120_mp3\train\positive already exist
Directory ..\..\dataset_based_p_y1120_mp3\train\neutral already exist
Directory ..\..\dataset_based_p_y1120_mp3\train\negative already exist
Directory ..\..\dataset_based_p_y1120_mp3\train\skip already exist
Directory ..\..\dataset_based_p_y1120_mp3\train\speech already exist
Directory ..\..\dataset_based_p_y1120_mp3\train\unknown already exist
Directory ..\..\dataset_based_p_y1120_mp3\val already exist
Directory ..\..\dataset_based_p_y1120_mp3\val\positive already exist
Directory ..\..\dataset_based_p_y1120_mp3\val\neutral already exist
Directory ..\..\dataset_based_p_y1120_mp3\val\negative already exist
Directory ..\..\dataset_based_p_y1120_mp3\val\skip already exist
Directory ..\..\dataset_based_p_y1120_mp3\val\speech already exist
Directory ..\..\dataset_based_p_y1120_mp3\val\unknown already exist
